# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = "../WeatherPy/city_output.csv"
df = pd.read_csv(city_data)
city_df = df.drop(columns="Unnamed: 0")
city_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Maitum,75,PH,2020-10-27 02:34:57,74,6.04,124.50,84.20,2.24
1,Boa Esperança,68,BR,2020-10-27 02:34:57,80,-21.09,-45.57,68.43,3.09
2,Cabo San Lucas,20,MX,2020-10-27 02:32:40,69,22.89,-109.91,82.40,11.41
3,Bluff,20,NZ,2020-10-27 02:32:19,64,-46.60,168.33,57.00,8.99
4,Atuona,49,PF,2020-10-27 02:32:19,76,-9.80,-139.03,78.78,22.35
...,...,...,...,...,...,...,...,...,...
559,Panzhihua,27,CN,2020-10-27 02:36:16,58,26.59,101.71,77.77,0.87
560,Sovetskiy,100,RU,2020-10-27 02:36:16,94,61.36,63.58,21.25,12.08
561,Laguna,1,US,2020-10-27 02:33:06,8,38.42,-121.42,69.01,12.75
562,Galveston,1,US,2020-10-27 02:32:07,78,29.37,-94.97,75.99,12.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key = g_key)

locations = city_df[["Latitude", "Longitude"]]
weights = city_df["Humidity"]

fig = gmaps.figure(map_type="SATELLITE", zoom_level=2, center=[0,0])
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, max_intensity=99, point_radius=10))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow DF to cities with Max Temp <80 and >70, Wind Speed < 10, Zero Cloudiness

cool = city_df.loc[(city_df["Max Temp"] < 80) & (city_df["Max Temp"] > 70) & (city_df["Wind Speed"] > 10) & (city_df["Cloudiness"] == 0)]
cool

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
99,Carnarvon,0,AU,2020-10-27 02:32:39,57,-24.87,113.63,77.00,25.28
110,Geraldton,0,AU,2020-10-27 02:32:32,47,-28.77,114.60,77.00,10.29
357,Benghazi,0,LY,2020-10-27 02:35:47,65,32.12,20.07,70.03,10.20
398,Aswān,0,EG,2020-10-27 02:29:13,31,24.09,32.91,74.28,10.13
459,Camargo,0,MX,2020-10-27 02:36:02,33,27.67,-105.17,72.93,22.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = cool
hotel_df["Hotel Name"] = ""

lat = hotel_df["Latitude"]
lng = hotel_df["Longitude"]
# print(lat, lng)

values = []

# for all numbers within range
for index, row in hotel_df.iterrows():
    coords=f"{row['Latitude']}, {row['Longitude']}"
    place_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"keyword": "hotel", "location": coords, "key": g_key, "radius": 5000}
    response = requests.get(place_url, params=params)
    data = response.json()
    name = data["results"][0]["name"]
    values.append(name)


/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
hotel_df["Hotel Name"] = values
hotel_df

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
99,Carnarvon,0,AU,2020-10-27 02:32:39,57,-24.87,113.63,77.00,25.28,The Gascoyne Hotel
110,Geraldton,0,AU,2020-10-27 02:32:32,47,-28.77,114.60,77.00,10.29,Ocean Centre Hotel
357,Benghazi,0,LY,2020-10-27 02:35:47,65,32.12,20.07,70.03,10.20,فندق نزل الراحة
398,Aswān,0,EG,2020-10-27 02:29:13,31,24.09,32.91,74.28,10.13,Sofitel Legend Old Cataract Aswan
459,Camargo,0,MX,2020-10-27 02:36:02,33,27.67,-105.17,72.93,22.30,Hotel Santa Fe


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations2, info_box_content=[row for row in hotel_info])
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))